# Task cell guide assign 
## Adding a method to assign cell guides to cell 
---
## Relevance:
Cells can have different sequencomg depht and suffer for another different covariate effects. We want to be able to take in consideration these fluctuations and assign cell guides to cells using some statistical method.

## Practical description:
- [x]  Create a “assign_guides” process  
- [ ] Use the Mudata ("mudata_results.h5mu") to capture the raw guide counts and the  cells covariates   
- [ ] Generate a binarized version of the counts matrix (0,1) 
- [ ] Add this the new matrix as a new mudata Layer ['binaryzed']
- [ ] Save the new Mudata

In [31]:
!pwd

/home/jovyan


In [11]:
%%writefile guide_assign.nf
nextflow.enable.dsl=2
params.MUDATA_INPUT = '/mnt/shared/gasperini_test_01/93/bc121425eff99125ff0ca316ceb0e7/raw_mudata_guide_and_transcripts.h5mu'  

process assign_guides {
    //execute the command to create this  env in the last cell block ("Install your Enviroment")
    conda '/home/jovyan/.conda/envs/hon_scanpy'
    
    debug true
    input:
    path mudata  
    output:
    path 'mu_with_binary.h5mu', emit : mu_with_binary  
    script:
    """
    chmod 700 /home/jovyan/assign_guide.py
    /home/jovyan/assign_guide.py ${mudata}
    """
    
}

workflow {
    
    assign_mu = assign_guides(params.MUDATA_INPUT)


}

Overwriting guide_assign.nf


In [12]:
!pwd

/home/jovyan


In [13]:
%%writefile assign_guide.py
#!/usr/bin/env python

import sys
import mudata as md
print ('-----')
print (sys.argv[1])

mudata = md.read(sys.argv[1])



print(mudata['guides'])
print(mudata['scRNA'])


#code to access the mudata['guide'] and binarize the matrix using your tool


#mudata.layers['binarized'] = your binarized matrix with same dimensions


mudata.write('mu_with_binary.h5mu')

Overwriting assign_guide.py


In [14]:
!nextflow run guide_assign.nf -w guide_assing_test  -with-conda  
# use -with-conda true        case using a custom conda enviroment


N E X T F L O W  ~  version 22.10.6
Launching `guide_assign.nf` [festering_pesquet] DSL2 - revision: 139f216f7c
[-        ] process > assign_guides -

executor >  local (1)
[4a/653ff8] process > assign_guides [  0%] 0 of 1

executor >  local (1)
[4a/653ff8] process > assign_guides [  0%] 0 of 1

executor >  local (1)
[4a/653ff8] process > assign_guides [100%] 1 of 1 ✔
-----
raw_mudata_guide_and_transcripts.h5mu
AnnData object with n_obs × n_vars = 5109 × 98
    obs: 'number_of_nonzero_guides', 'batch_number'
    var: 'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'
AnnData object with n_obs × n_vars = 5109 × 16094
    obs: 'n_genes', 'n_counts', 'percent_mito', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'batch_number'
    var: 'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end'




# Sandbox
This field contains some of the comands needed to acces the MuData  
Please check: Mudata and Anndata documentation  
obs: objects inside Mudata ex: mdata['guides'] is an anndata object  

In [22]:
import mudata as md

In [23]:
mdata = md.read_h5mu("/mnt/shared/gasperini_test_01/93/bc121425eff99125ff0ca316ceb0e7/raw_mudata_guide_and_transcripts.h5mu")

In [24]:
mdata

MuData object with n_obs × n_vars = 5109 × 16192
  var:	'feature_name'
  2 modalities
    guides:	5109 x 98
      obs:	'number_of_nonzero_guides', 'batch_number'
      var:	'feature_name', 'guide_chr', 'guide_end', 'guide_start', 'guide_number', 'target_elements'
    scRNA:	5109 x 16094
      obs:	'n_genes', 'n_counts', 'percent_mito', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'batch_number'
      var:	'feature_name', 'n_cells', 'transcript_chr', 'transcript_start', 'transcript_end'

In [25]:
#capturing guides
mdata['guides'].X

<5109x98 sparse matrix of type '<class 'numpy.float32'>'
	with 20102 stored elements in Compressed Sparse Row format>

In [29]:
mdata.var.head()   #showing vars

,guides:guide_chr,guides:guide_end,guides:guide_start,guides:guide_number,guides:target_elements,scRNA:n_cells,scRNA:transcript_chr,scRNA:transcript_start,scRNA:transcript_end,feature_name
BROX|1_sgrna_chr1:222712600:222712620,chr1,222712620,222712600,1,BROX,NaN,NaN,NaN,NaN,BROX|1_sgrna_chr1:222712600:222712620
BROX|2_sgrna_chr1:222712744:222712764,chr1,222712764,222712744,2,BROX,NaN,NaN,NaN,NaN,BROX|2_sgrna_chr1:222712744:222712764
CCND2|1_sgrna_chr12:4273763:4273783,chr12,4273783,4273763,1,CCND2,NaN,NaN,NaN,NaN,CCND2|1_sgrna_chr12:4273763:4273783
CCND2|2_sgrna_chr12:4273832:4273852,chr12,4273852,4273832,2,CCND2,NaN,NaN,NaN,NaN,CCND2|2_sgrna_chr12:4273832:4273852
CEBPB|1_sgrna_chr20:50190830:50190850,chr20,50190850,50190830,1,CEBPB,NaN,NaN,NaN,NaN,CEBPB|1_sgrna_chr20:50190830:50190850


In [30]:
mdata.obs.head()   #showing covariates

,guides:number_of_nonzero_guides,guides:batch_number,scRNA:n_genes,scRNA:n_counts,scRNA:percent_mito,scRNA:doublet_scores,scRNA:predicted_doublets,scRNA:doublet_info,scRNA:batch_number
barcode,,,,,,,,,
AAACCTGAGAGGGCTT,1,1,248,325.0,0.000000,0.022133,False,False,1
AAACCTGCAAATACAG,2,1,3504,8941.0,0.022816,0.116685,False,False,1
AAACCTGCACTCGACG,2,1,2403,7169.0,0.051611,0.070968,False,False,1
AAACCTGCAGTCTTCC,3,1,2791,8174.0,0.053218,0.064220,False,False,1
AAACCTGCATGCCTAA,4,1,5072,23770.0,0.045982,0.142857,False,False,1


In [28]:
mdata.var["guides:target_elements"].value_counts()

BROX        2
OARD1       2
PCBD1       2
PLEKHJ1     2
PPID        2
PPP1R11     2
PSMG2       2
RBM17       2
RNF181      2
RPS3        2
RWDD1       2
SDHC        2
SEPT11      2
SLC25A4     2
SMARCC1     2
SPCS2       2
SRRM1       2
ST13        2
TMEM167A    2
TMEM242     2
TSPO        2
UAP1        2
UBE2T       2
ODC1        2
NEK2        2
CCND2       2
NDUFB7      2
CEBPB       2
CHRAC1      2
CITED2      2
COMT        2
DECR1       2
EBPL        2
EIF4EBP1    2
FAM83A      2
FECH        2
GATA2       2
GYPC        2
HBD         2
HNRNPA1     2
ITGB1BP1    2
JAGN1       2
LAMTOR1     2
LMO2        2
LRIF1       2
MAN1A1      2
MRPS35      2
MRPS36      2
VDAC1       2
Name: guides:target_elements, dtype: int64

In [ ]:
# subsetting
mdata[:,mdata.var["guides:target_elements"] == "BROX"]

In [ ]:
# encode gene-guide relationships


# Install your Enviroment

In [ ]:
%%writefile hon_guide.yaml
name: hon_scanpy
channels:
  - conda-forge
  - defaults
dependencies:
  - _libgcc_mutex=0.1=main
  - _openmp_mutex=5.1=1_gnu
  - backcall=0.2.0=pyhd3eb1b0_0
  - blas=1.0=mkl
  - blosc=1.21.0=h4ff587b_1
  - bottleneck=1.3.4=py37hce1f21e_0
  - brotli=1.0.9=he6710b0_2
  - bzip2=1.0.8=h7b6447c_0
  - ca-certificates=2022.12.7=ha878542_0
  - cairo=1.16.0=hcf35c78_1003
  - certifi=2022.12.7=pyhd8ed1ab_0
  - cycler=0.11.0=pyhd3eb1b0_0
  - debugpy=1.5.1=py37h295c915_0
  - decorator=5.1.1=pyhd3eb1b0_0
  - entrypoints=0.4=py37h06a4308_0
  - fontconfig=2.13.1=h86ecdb6_1001
  - fonttools=4.25.0=pyhd3eb1b0_0
  - freetype=2.11.0=h70c0345_0
  - giflib=5.2.1=h7b6447c_0
  - glib=2.68.2=h36276a3_0
  - gmp=6.2.1=h58526e2_0
  - hdf5=1.10.4=hb1b8bf9_0
  - icu=64.2=he1b5a44_1
  - intel-openmp=2021.4.0=h06a4308_3561
  - ipykernel=6.15.2=py37h06a4308_0
  - ipython=7.31.1=py37h06a4308_1
  - jedi=0.18.1=py37h06a4308_1
  - joblib=1.1.1=py37h06a4308_0
  - jpeg=9e=h7f8727e_0
  - jupyter_client=7.2.2=py37h06a4308_0
  - jupyter_core=4.11.2=py37h06a4308_0
  - kiwisolver=1.4.2=py37h295c915_0
  - lcms2=2.12=h3be6417_0
  - ld_impl_linux-64=2.38=h1181459_1
  - leidenalg=0.8.2=py37h43df1e8_0
  - libffi=3.3=he6710b0_2
  - libgcc-ng=9.1.0=hdf63c60_0
  - libgfortran-ng=7.3.0=hdf63c60_0
  - libgomp=11.2.0=h1234567_1
  - libiconv=1.16=h516909a_0
  - libllvm10=10.0.1=hbcb73fb_5
  - libpng=1.6.37=hbc83047_0
  - libsodium=1.0.18=h7b6447c_0
  - libstdcxx-ng=9.1.0=hdf63c60_0
  - libtiff=4.2.0=h2818925_1
  - libuuid=2.32.1=h14c3975_1000
  - libwebp=1.2.2=h55f646e_0
  - libwebp-base=1.2.2=h7f8727e_0
  - libxcb=1.13=h14c3975_1002
  - libxml2=2.9.10=hee79883_0
  - llvmlite=0.35.0=py37h612dafd_4
  - lz4-c=1.9.3=h295c915_1
  - lzo=2.10=h7b6447c_2
  - matplotlib-base=3.5.1=py37ha18d171_1
  - matplotlib-inline=0.1.6=py37h06a4308_0
  - mkl=2021.4.0=h06a4308_640
  - mkl-service=2.4.0=py37h7f8727e_0
  - mkl_fft=1.3.1=py37hd3c417c_0
  - mkl_random=1.2.2=py37h51133e4_0
  - mock=4.0.3=pyhd3eb1b0_0
  - munkres=1.1.4=py_0
  - ncurses=6.3=h7f8727e_2
  - nest-asyncio=1.5.6=py37h06a4308_0
  - numba=0.52.0=py37ha9443f7_0
  - numexpr=2.8.1=py37h6abb31d_0
  - numpy=1.21.5=py37he7a7128_2
  - numpy-base=1.21.5=py37hf524024_2
  - openssl=1.1.1t=h7f8727e_0
  - packaging=22.0=py37h06a4308_0
  - pandas=1.3.5=py37h8c16a72_0
  - parso=0.8.3=pyhd3eb1b0_0
  - patsy=0.5.3=py37h06a4308_0
  - pcre=8.44=he1b5a44_0
  - pexpect=4.8.0=pyhd3eb1b0_3
  - pickleshare=0.7.5=pyhd3eb1b0_1003
  - pillow=9.0.1=py37h22f2fdc_0
  - pip=22.3.1=py37h06a4308_0
  - pixman=0.38.0=h516909a_1003
  - prompt-toolkit=3.0.36=py37h06a4308_0
  - psutil=5.8.0=py37h27cfd23_1
  - pthread-stubs=0.4=h36c2ea0_1001
  - ptyprocess=0.7.0=pyhd3eb1b0_2
  - pycairo=1.20.0=py37h01af8b0_1
  - pygments=2.11.2=pyhd3eb1b0_0
  - pyparsing=3.0.9=py37h06a4308_0
  - pytables=3.6.1=py37h71ec239_0
  - python=3.7.13=h12debd9_0
  - python-dateutil=2.8.2=pyhd3eb1b0_0
  - python-igraph=0.8.2=py37hb78272c_0
  - python_abi=3.7=2_cp37m
  - pytz=2022.7=py37h06a4308_0
  - pyzmq=22.3.0=py37h295c915_2
  - readline=8.1.2=h7f8727e_1
  - scikit-learn=1.0.2=py37h51133e4_1
  - scipy=1.6.2=py37had2a1c9_1
  - seaborn=0.12.2=py37h06a4308_0
  - setuptools=65.6.3=py37h06a4308_0
  - six=1.16.0=pyhd3eb1b0_1
  - sqlite=3.38.5=hc218d9a_0
  - statsmodels=0.13.2=py37h7f8727e_0
  - texttable=1.6.7=pyhd8ed1ab_0
  - threadpoolctl=2.2.0=pyh0d69192_0
  - tk=8.6.12=h1ccaba5_0
  - tornado=6.1=py37h27cfd23_0
  - traitlets=5.7.1=py37h06a4308_0
  - typing_extensions=4.3.0=py37h06a4308_0
  - wcwidth=0.2.5=pyhd3eb1b0_0
  - wheel=0.38.4=py37h06a4308_0
  - xorg-kbproto=1.0.7=h14c3975_1002
  - xorg-libice=1.0.10=h516909a_0
  - xorg-libsm=1.2.3=h84519dc_1000
  - xorg-libx11=1.6.12=h36c2ea0_0
  - xorg-libxau=1.0.9=h14c3975_0
  - xorg-libxdmcp=1.1.3=h516909a_0
  - xorg-libxext=1.3.4=h516909a_0
  - xorg-libxrender=0.9.10=h516909a_1002
  - xorg-renderproto=0.11.1=h14c3975_1002
  - xorg-xextproto=7.3.0=h14c3975_1002
  - xorg-xproto=7.0.31=h14c3975_1007
  - xz=5.2.5=h7f8727e_1
  - zeromq=4.3.4=h2531618_0
  - zlib=1.2.12=h7f8727e_2
  - zstd=1.5.2=ha4553b6_0
  - pip:
    - mudata
    - anndata==0.8.0
    - cython==0.29.33
    - deprecated==1.2.13
    - dnaio==0.10.0
    - dunamai==1.16.0
    - fba==0.0.13
    - future==0.18.3
    - get-version==3.5.4
    - graphtools==1.5.3
    - h5py==3.8.0
    - hdbscan==0.8.29
    - igraph==0.10.4
    - importlib-metadata==6.1.0
    - isal==1.1.0
    - legacy-api-wrap==1.2
    - louvain==0.8.0
    - natsort==7.0.0
    - networkx==2.6.3
    - parsedatetime==2.6
    - phate==1.0.10
    - polyleven==0.8
    - pybktree==1.1
    - pyclustering==0.10.1.2
    - pygsp==0.5.1
    - pysam==0.20.0
    - regex==2022.10.31
    - s-gd2==1.8.1
    - scanpy==1.7.0rc1
    - scprep==1.2.2
    - sinfo==0.3.4
    - stdlib-list==0.8.0
    - tasklogger==1.2.0
    - tqdm==4.65.0
    - umap-learn==0.4.6
    - umi-tools==1.1.4
    - wrapt==1.15.0
    - xopen==1.7.0
    - zipp==3.15.0

In [ ]:
!mamba env create -f hon_guide.yaml --force

In [5]:
!conda env list

# conda environments:
#
hon_scanpy               /home/jovyan/.conda/envs/hon_scanpy
base                     /opt/conda
pygenomictracks          /opt/conda/envs/pygenomictracks



In [ ]:
#Use this env  "/home/jovyan/.conda/envs/hon_scanpy"